In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import html
import json
import time
from requests.exceptions import ConnectionError
import unicodedata
#import mysql.connector
import math

## Seluruh Link Nama Mobil yang mau dicari reviewnya

In [7]:
list_mobil_seva=pd.read_csv('Links Mobil OTO Review - Sheet1.csv')

In [14]:
list_mobil_seva.head()

,Brand,Model,Links
0,Chevrolet,Captiva,https://www.oto.com/mobil-baru/chevrolet/capti...
1,Chevrolet,Spin,NaN
2,Daihatsu,Ayla,https://www.oto.com/mobil-baru/daihatsu/ayla/r...
3,Daihatsu,Espass,NaN
4,Daihatsu,Gran Max,https://www.oto.com/mobil-baru/daihatsu/gran-m...


In [4]:
mobil=list_mobil_seva['Links'].dropna().tolist()

## Scrap Seluruh Link Comment Satu Mobil

In [115]:
def ambil_link(urls, listlink, page):
    url = "%s?page=%s"%(urls, str(page))
#     url = link+'?page='+str(page)
#    print(url)
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html5lib")
    find_link = soup.find('ul', class_="reviewlist")
    links = find_link.findAll('a', href=True)
    
    for link in links:
        if 'lihat lainnya' not in link.get_text(strip=True).lower():
            listlink.append(link['href'])
    
    scripts = json.loads(soup.findAll('script', {'type':'application/ld+json'})[-1].get_text(strip=True))
    angka_max = scripts['aggregateRating']['reviewCount']
    max_page=math.ceil(angka_max/10)
    if page < max_page:
        listlink = ambil_link(urls, listlink, page+1)
    return listlink

In [117]:
articles1 = {}
for i in mobil:
    url = i
    page=1
    articleid = url.split('/')
    articles1['Model'] = articleid[4]+' '+articleid[5].replace('-',' ')
    k=articles1.popitem()[1]
    articles1[k] = ambil_link(url,[],page)

## Scrap Per Comment Satu Mobil

In [119]:
def baca_review(mobil,url):
    articles = {}
    url =url
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html5lib")
    
    #extract comment_content
    comment_content = soup.find('div', class_="user-content").get_text(strip=True)
    articles['comment_content'] = comment_content
    
    #Extract rating Comment
    comment_rating = soup.find('span', class_="hightligtht").get_text(strip=True)
    articles['comment_rating'] = comment_rating
    
    #Extract helping impact
    helping_impact = soup.find('span', class_="like-hint helpful-yes-count").get_text(strip=True)
    articles['Likes_count'] = helping_impact
    
    #extract waktu
    view=soup.find('ul',class_="published-info clearfix")
    comment_date = view.findAll('li')[-3].get_text(strip=True) if view else ''
    articles['comment_date'] = comment_date
    
    #Extract title Comment
    find_title = soup.find('main', class_="modelpage page")
    comment_title = find_title.find('h1').get_text(strip=True) if find_title else ''
    articles['comment_title'] = comment_title
    
    #Extract number seen
    view=soup.find('ul',class_="published-info clearfix")
    comment_seen = view.findAll('li')[-2].get_text(strip=True) if view else ''
    articles['comment_seen'] = comment_seen
    
    #extract Name
    scripts = json.loads(soup.findAll('script', {'type':'application/ld+json'})[-1].get_text(strip=True))
    name = scripts['author']['name']
    articles['Name'] = name
    
    articles['Nama Mobil'] = mobil
    
    articles['Link'] = url
    
    return articles

In [120]:
semua_rev = []
for key,value in articles1.items():
        for j in range(0,len(value)):
            if value[j]!='javascript:void(0);':
                articles=baca_review(key,value[j])
                semua_rev.append(articles)

In [121]:
pd.DataFrame(semua_rev)

,Likes_count,Link,Nama Mobil,Name,comment_content,comment_date,comment_rating,comment_seen,comment_title
0,12,https://www.oto.com/mobil-baru/chevrolet/capti...,chevrolet captiva,ervandy afianto,mobil chevrolet ini sangat tangguh dan kaki2 s...,"Apr 28, 2018",5/5,418 Dilihat,mobil idaman keluarga Review Pengguna
1,16,https://www.oto.com/mobil-baru/chevrolet/capti...,chevrolet captiva,David,Mobil yang ganteng dan tidak bosan menurut say...,"Feb 04, 2018",4/5,782 Dilihat,Ganteng tetapi sedikit boros di putaran bawah ...
2,16,https://www.oto.com/mobil-baru/chevrolet/capti...,chevrolet captiva,Taufik Rahman,Mobil saya caftiva Bensin manual tahun 2010 su...,"Jan 03, 2018",5/5,484 Dilihat,Badak Review Pengguna
3,9,https://www.oto.com/mobil-baru/daihatsu/ayla/r...,daihatsu ayla,Kurniawan,"Mobil nya Simpel,,Elegan,,enak di pakai dan co...","Agt 29, 2018",5/5,675 Dilihat,Curahan hati ( JUJUR ) Review Pengguna
4,12,https://www.oto.com/mobil-baru/daihatsu/ayla/r...,daihatsu ayla,Ridho Alfianto,"Mobil idaman para keluarga Indonesia,irit,bert...","Jun 19, 2018",5/5,858 Dilihat,Mobil irit dan gesit Review Pengguna
5,6,https://www.oto.com/mobil-baru/daihatsu/ayla/r...,daihatsu ayla,Resky putra Nasution,Mobil yang sesuai dengan kondisi jalanan kota ...,"Mei 15, 2018",5/5,571 Dilihat,"Simpel , aman , bisa masuk ke jalan kecil Revi..."
6,7,https://www.oto.com/mobil-baru/daihatsu/ayla/r...,daihatsu ayla,Rinaldi Elektrische,AYLA 1.2 X AT ambil pas akhir tahun 2017... se...,"Mei 14, 2018",5/5,1236 Dilihat,"ELEGAN, GACOR, GESIT DAN IRIT Review Pengguna"
7,9,https://www.oto.com/mobil-baru/daihatsu/ayla/r...,daihatsu ayla,Eric Valrntin,"Saya menggunakan Daihatsu Ayla tahun 2015, awa...","Mei 07, 2018",4/5,922 Dilihat,Mobil Mungil Tapi Luas Review Pengguna
8,5,https://www.oto.com/mobil-baru/daihatsu/ayla/r...,daihatsu ayla,Eko Rudiyanyo,Walaupun kecil tapi pas buat keluarga kecilnya...,"Apr 24, 2018",5/5,475 Dilihat,Siputih Mungil Irit Review Pengguna
9,4,https://www.oto.com/mobil-baru/daihatsu/ayla/r...,daihatsu ayla,Yusak andy,"Mobil murah, tapi tidak murahan.bodynya kecil ...","Apr 07, 2018",5/5,619 Dilihat,Kecil2 cabe rawit Review Pengguna


In [122]:
hasil_scrapping_oto=pd.DataFrame(semua_rev)
hasil_scrapping_oto['Nama Mobil'].value_counts()

toyota avanza                61
daihatsu xenia               46
honda mobilio                35
toyota kijang innova         34
toyota rush                  28
toyota calya                 26
honda brio                   26
nissan grand livina          23
toyota avanza veloz          22
daihatsu ayla                20
honda jazz                   20
suzuki sx 4 s cross          17
daihatsu terios              15
toyota agya                  15
honda hr v                   14
toyota sienta                13
honda civic                  11
mitsubishi mirage             9
suzuki karimun wagon r        8
suzuki ertiga                 8
toyota fortuner               8
honda cr v                    7
daihatsu sirion               6
kia picanto                   5
nissan march                  4
daihatsu luxio                3
daihatsu gran max mb          3
mazda 2                       3
chevrolet captiva             3
suzuki karimun wagon r gs     3
mazda cx 3                    2
toyota e

In [123]:
hasil_scrapping_oto.to_csv('hasil_scrapping_oto.csv')